In [ ]:
import numpy as np

### Example
* Suppose we have the following optimization problem:
$$
\text{maximize} \quad x_1 + 2x_2 + 4x_3 \\
\begin{aligned}
3x_1 + x_2 + 5x_3 &\leq 10 \\
x_1 + 4x_2 + x_3 &\leq 8 \\
2x_1 + 0x_2 + 2x_3 &\leq 7 \\
\end{aligned}
$$

* The standard form of this optimization problem is given as:
$$
\text{maximize} \quad x_1 + 2x_2 + 4x_3 + 0x_4 + 0x_ 5 + 0x_6 \\
\begin{aligned}
3x_1 + x_2 + 5x_3 + x_4 &= 10 \\
x_1 + 4x_2 + x_3 + x_5 &= 8 \\
2x_1 + 0x_2 + 2x_3 + x_6 &= 7 \\
\end{aligned}
$$

* We can rewrite this optimization problem as follows:
$$
\text{minimize} \quad -x_1 - 2x_2 - 4x_3 - 0x_4 - 0x_ 5 - 0x_6 \\
\begin{aligned}
3x_1 + x_2 + 5x_3 + x_4 &= 10 \\
x_1 + 4x_2 + x_3 + x_5 &= 8 \\
2x_1 + 0x_2 + 2x_3 + x_6 &= 7 \\
\end{aligned}

$$

* We can express this optimization problem with the following matrix notation:
$$
\text{minimize} \quad \mathbf{d}^T\mathbf{x}
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$
with $\mathbf{d} = \begin{bmatrix} -1 \\ -2 \\ -4 \\ 0 \\ 0 \\ 0 \end{bmatrix}$, $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \end{bmatrix}$,
$\mathbf{A} = \begin{bmatrix} 3 & 1 & 5 & 1 & 0 & 0 \\ 1 & 4 & 1 & 0 & 1 & 0 \\ 2 & 0 & 2 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0  \\ 0 & 0 & 0 & 0 & 0 & 0  \end{bmatrix}$, $\mathbf{b} = \begin{bmatrix} 10 \\ 8 \\ 7 \\ 0 \\ 0 \\ 0 \end{bmatrix}$

# Implementation of ADMM
## Overview
* Given the following optimization problem:
$$
\text{minimize} \quad \mathbf{d}^T\mathbf{x} \\
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

* We can formulate it as:
$$
\text{minimize} \quad \frac{\rho}{2}||\mathbf{x} - \alpha||^2_2
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

with $\rho = 1$, $\mathbf{A} = \begin{bmatrix} 3. & 1. & 5. \\ 1. & 4. & 1. \\ 2. & 0. & 2. \end{bmatrix}$, $\mathbf{b} = \begin{bmatrix} 10. \\ 8. \\ 7. \end{bmatrix}$, and $\mathbf{d} = \begin{bmatrix} -1 \\ -2 \\ -4 \end{bmatrix}$

* Step 1: Construct the construction matrix $\mathbf{C} = \begin{bmatrix} \rho\mathbf{I} & \mathbf{A}^T \\ \mathbf{A} & 0  \end{bmatrix}$
* Step 2: Construct the vector $\mathbf{m} = \begin{bmatrix} \rho\pmb{\alpha} \\ \mathbf{b} \end{bmatrix}$ with $\pmb{\alpha}:= \mathbf{y}^k - (1/\rho)(\pmb{\mu}^k + \mathbf{d})$ at $k = 0$
* Step 3: Solve the equation $\mathbf{C}\begin{bmatrix}\mathbf{x} \\ \pmb{\lambda}\end{bmatrix}= \mathbf{m}$

In [ ]:
d = np.array([-1., -2., -4., 0, 0, 0]).T
A = np.array([[3., 1., 5., 1 , 0 , 0], [1., 4., 1., 0 ,1, 0], [2., 0., 2., 0, 0 ,1]])
b = np.array([10, 8, 7]).T
rho = 1

no_of_vars = A.shape[1]
no_of_constraints = A.shape[0]

print("Problem Size: constraints (rows) = {}; variables (columns) = {}".format(no_of_constraints,no_of_vars))

I = np.identity(no_of_vars)
zeros = np.zeros(shape = (no_of_constraints,no_of_constraints))
C = np.block([[rho * I, A.T], [A, zeros]])
print(C)
MAX_ITER = 100
y = 0*np.random.normal(size = (1,no_of_vars))
mu = 0*np.random.normal(size = (1,no_of_vars))

for i in range(MAX_ITER):
    ### Solve for x
    alpha = y - (1/rho) * (mu + d)

    m = np.block([rho * alpha, b])

    x_lambda = np.linalg.solve(C, m.T)

    print(np.allclose(np.dot(C, x_lambda), m.T))

    x = x_lambda[:6]

    ### Solve for y:
    beta = x + (1/rho) * mu.T
    beta[beta < 0] = 0
    y = beta.T

    ### Solve for mu:
    mu1 = rho * (x.T - y)
    mu = mu + mu1

    print(f"{x.T}\n");



[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

[[10.]
 [ 8.]
 [ 7.]]

